# Data and Library Consolidation

## Import Libraries used in notebook

In [1]:
import pandas as pd # For working with dataframes
from sklearn import decomposition # For performing PCA on the data
from sklearn.model_selection import train_test_split #Data splitting
from sklearn.ensemble import RandomForestClassifier #Random Forest Classifier for training
import matplotlib.pyplot as plt # For plotting visualizations of analysis
import altair as alt # For additional plotting functionality
import warnings # Suppress warnings from output
from sklearn.multioutput import MultiOutputClassifier #For prediction into multiple categories
from sklearn.metrics import classification_report, accuracy_score #Assessing the model
from sklearn.pipeline import Pipeline #Create a pipeline of all of the data processing steps

## Import and look at the snps data for the samples

In [2]:
df = pd.read_csv('matrix_10step.csv')
df

,Unnamed: 0,rs79725552,rs4965019,rs147891127,rs144535625,rs3013006,rs150314208,rs151049930,rs145922255,rs144961645,...,rs186168008,rs76759269,rs182890688,rs148355237,rs189454695,rs145777112,rs191731586,rs141330630,rs201907533,Population code
0,HG00096,0,1,0,0,1,0,0,0,0,...,1,2,0,0,1,0,0,0,0,GBR
1,HG00097,1,1,1,0,0,1,1,1,0,...,1,2,0,0,0,0,0,0,0,GBR
2,HG00099,1,0,1,0,0,1,0,1,0,...,0,2,0,0,0,0,0,0,0,GBR
3,HG00100,0,0,0,0,1,0,0,0,0,...,0,2,0,0,0,0,0,0,0,GBR
4,HG00101,1,0,0,0,1,1,1,1,0,...,1,2,0,0,0,0,0,0,0,GBR
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1087,NA20816,0,0,0,0,1,0,0,0,0,...,0,2,0,0,1,0,0,0,0,TSI
1088,NA20818,0,0,0,0,0,0,0,0,0,...,0,2,0,0,1,0,0,0,0,TSI
1089,NA20819,0,0,0,0,0,0,0,0,0,...,0,2,0,0,1,0,0,0,0,TSI
1090,NA20826,0,0,0,0,0,0,0,0,0,...,0,2,0,0,1,0,0,0,0,TSI


## Provide the appropriate label for the unnamed column --> Samples. The rest of the columns refer to the snps

In [3]:
df.rename(columns={'Unnamed: 0': 'Sample'}, inplace=True) #The columns correspond to the specific snp that is present (1) or absent (0)
df.head()

,Sample,rs79725552,rs4965019,rs147891127,rs144535625,rs3013006,rs150314208,rs151049930,rs145922255,rs144961645,...,rs186168008,rs76759269,rs182890688,rs148355237,rs189454695,rs145777112,rs191731586,rs141330630,rs201907533,Population code
0,HG00096,0,1,0,0,1,0,0,0,0,...,1,2,0,0,1,0,0,0,0,GBR
1,HG00097,1,1,1,0,0,1,1,1,0,...,1,2,0,0,0,0,0,0,0,GBR
2,HG00099,1,0,1,0,0,1,0,1,0,...,0,2,0,0,0,0,0,0,0,GBR
3,HG00100,0,0,0,0,1,0,0,0,0,...,0,2,0,0,0,0,0,0,0,GBR
4,HG00101,1,0,0,0,1,1,1,1,0,...,1,2,0,0,0,0,0,0,0,GBR


## Import population data

In [4]:
pop_df = pd.read_csv('igsr_populations.tsv', sep='\t')
print(f'Size of df: {pop_df.shape}')
pop_df.head()

Size of df: (212, 11)


,Population code,Population elastic ID,Population name,Population description,Population latitude,Population longitude,Superpopulation code,Superpopulation name,Superpopulation display colour,Superpopulation display order,Data collections
0,CHS,CHS,Southern Han Chinese,Han Chinese South,23.13333,113.266667,EAS,East Asian Ancestry,#778500,3,"1000 Genomes on GRCh38,1000 Genomes 30x on GRC..."
1,KHV,KHV,Kinh Vietnamese,"Kinh in Ho Chi Minh City, Vietnam",10.78000,106.680000,EAS,East Asian Ancestry,#778500,3,"1000 Genomes on GRCh38,1000 Genomes 30x on GRC..."
2,NaN,BrahminSGDP,Brahmin,Brahmin in India (SGDP),17.70000,83.300000,NaN,South Asia (SGDP),#008c1e,18,Simons Genome Diversity Project
3,NaN,MiaoSGDP,Miao,Miao in China (SGDP),28.00000,109.000000,NaN,East Asia (SGDP),#ff48de,16,Simons Genome Diversity Project
4,NaN,KyrgyzSGDP,Kyrgyz,Kyrgyz in Kyrgyzystan (SGDP),42.90000,74.600000,NaN,Central Asia and Siberia (SGDP),#01daa0,15,Simons Genome Diversity Project


## Separate data into training. validation and test sets

In [5]:
train_set, testing_set = train_test_split(df, test_size = 0.3, stratify = df['Population code'])
validation_test, test_set = train_test_split(testing_set, test_size = 0.5, stratify = testing_set['Population code'])
print(train_set.shape)
train_set.head()

(764, 49434)


,Sample,rs79725552,rs4965019,rs147891127,rs144535625,rs3013006,rs150314208,rs151049930,rs145922255,rs144961645,...,rs186168008,rs76759269,rs182890688,rs148355237,rs189454695,rs145777112,rs191731586,rs141330630,rs201907533,Population code
284,HG00705,0,0,0,0,2,0,0,0,0,...,0,2,0,0,1,0,0,1,0,CHS
310,HG01083,0,0,0,0,0,0,0,0,0,...,0,2,0,0,1,0,0,0,0,PUR
743,NA19119,0,0,0,0,1,0,0,0,0,...,0,2,0,0,0,0,0,0,0,YRI
690,NA19010,0,0,0,0,2,0,0,0,0,...,0,2,0,0,0,0,0,0,0,JPT
130,HG00319,0,0,0,0,0,0,0,0,0,...,0,2,0,0,1,0,0,0,0,FIN


# Data Processing Steps

## Add the demographical data

In [6]:
train_set = train_set.merge(pop_df, on='Population code', how='inner') # Merge the demographical data  to the df with principal component values
print(f'Size of df: {train_set.shape}')
train_set.head()

Size of df: (764, 49444)


,Sample,rs79725552,rs4965019,rs147891127,rs144535625,rs3013006,rs150314208,rs151049930,rs145922255,rs144961645,...,Population elastic ID,Population name,Population description,Population latitude,Population longitude,Superpopulation code,Superpopulation name,Superpopulation display colour,Superpopulation display order,Data collections
0,HG00705,0,0,0,0,2,0,0,0,0,...,CHS,Southern Han Chinese,Han Chinese South,23.13333,113.266667,EAS,East Asian Ancestry,#778500,3,"1000 Genomes on GRCh38,1000 Genomes 30x on GRC..."
1,HG01083,0,0,0,0,0,0,0,0,0,...,PUR,Puerto Rican,Puerto Rican in Puerto Rico,18.40000,-66.100000,AMR,American Ancestry,#710027,2,"1000 Genomes on GRCh38,1000 Genomes 30x on GRC..."
2,NA19119,0,0,0,0,1,0,0,0,0,...,YRI,Yoruba,"Yoruba in Ibadan, Nigeria",7.40000,3.920000,AFR,African Ancestry,#ffd845,1,"1000 Genomes on GRCh38,1000 Genomes 30x on GRC..."
3,NA19010,0,0,0,0,2,0,0,0,0,...,JPT,Japanese,"Japanese in Tokyo, Japan",35.68000,139.680000,EAS,East Asian Ancestry,#778500,3,"1000 Genomes on GRCh38,1000 Genomes 30x on GRC..."
4,HG00319,0,0,0,0,0,0,0,0,0,...,FIN,Finnish,Finnish in Finland,60.17000,24.930000,EUR,European Ancestry,#018ead,4,"1000 Genomes on GRCh38,1000 Genomes 30x on GRC..."


## Separate the categorical and numerical data

In [7]:
# Select all numerical columns
snps_df = train_set.select_dtypes(include=['number'])
meta_df = train_set.select_dtypes(exclude=['number'])
print(snps_df.shape)
meta_df.head()

(764, 49435)


,Sample,Population code,Population elastic ID,Population name,Population description,Superpopulation code,Superpopulation name,Superpopulation display colour,Data collections
0,HG00705,CHS,CHS,Southern Han Chinese,Han Chinese South,EAS,East Asian Ancestry,#778500,"1000 Genomes on GRCh38,1000 Genomes 30x on GRC..."
1,HG01083,PUR,PUR,Puerto Rican,Puerto Rican in Puerto Rico,AMR,American Ancestry,#710027,"1000 Genomes on GRCh38,1000 Genomes 30x on GRC..."
2,NA19119,YRI,YRI,Yoruba,"Yoruba in Ibadan, Nigeria",AFR,African Ancestry,#ffd845,"1000 Genomes on GRCh38,1000 Genomes 30x on GRC..."
3,NA19010,JPT,JPT,Japanese,"Japanese in Tokyo, Japan",EAS,East Asian Ancestry,#778500,"1000 Genomes on GRCh38,1000 Genomes 30x on GRC..."
4,HG00319,FIN,FIN,Finnish,Finnish in Finland,EUR,European Ancestry,#018ead,"1000 Genomes on GRCh38,1000 Genomes 30x on GRC..."


# Create models for training and fit it to the data

In [8]:
rfclass = RandomForestClassifier() # n_estimators=100,max_features='sqrt',max_depth=None,min_samples_split=2,min_samples_leaf=1,bootstrap=True,random_state=42,class_weight='balanced'
rfclass

RandomForestClassifier()

## Create Dummies for the Superpopulation categories using One-Hot Encoding

In [9]:
meta_df = pd.get_dummies(meta_df, columns=['Superpopulation name'])

In [10]:
meta_df.head()

,Sample,Population code,Population elastic ID,Population name,Population description,Superpopulation code,Superpopulation display colour,Data collections,Superpopulation name_African Ancestry,Superpopulation name_American Ancestry,Superpopulation name_East Asian Ancestry,Superpopulation name_European Ancestry
0,HG00705,CHS,CHS,Southern Han Chinese,Han Chinese South,EAS,#778500,"1000 Genomes on GRCh38,1000 Genomes 30x on GRC...",False,False,True,False
1,HG01083,PUR,PUR,Puerto Rican,Puerto Rican in Puerto Rico,AMR,#710027,"1000 Genomes on GRCh38,1000 Genomes 30x on GRC...",False,True,False,False
2,NA19119,YRI,YRI,Yoruba,"Yoruba in Ibadan, Nigeria",AFR,#ffd845,"1000 Genomes on GRCh38,1000 Genomes 30x on GRC...",True,False,False,False
3,NA19010,JPT,JPT,Japanese,"Japanese in Tokyo, Japan",EAS,#778500,"1000 Genomes on GRCh38,1000 Genomes 30x on GRC...",False,False,True,False
4,HG00319,FIN,FIN,Finnish,Finnish in Finland,EUR,#018ead,"1000 Genomes on GRCh38,1000 Genomes 30x on GRC...",False,False,False,True


## Define X_train and y_train

In [11]:
X_train = snps_df.drop(['Population latitude','Population longitude','Superpopulation display order'], axis = 1)
X_train

,rs79725552,rs4965019,rs147891127,rs144535625,rs3013006,rs150314208,rs151049930,rs145922255,rs144961645,rs144366698,...,rs6010087,rs186168008,rs76759269,rs182890688,rs148355237,rs189454695,rs145777112,rs191731586,rs141330630,rs201907533
0,0,0,0,0,2,0,0,0,0,0,...,0,0,2,0,0,1,0,0,1,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,2,0,0,1,0,0,0,0
2,0,0,0,0,1,0,0,0,0,0,...,0,0,2,0,0,0,0,0,0,0
3,0,0,0,0,2,0,0,0,0,0,...,0,0,2,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,2,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
759,1,1,1,0,0,0,0,1,0,0,...,0,1,2,0,0,0,0,0,0,0
760,0,0,0,0,1,0,0,0,0,0,...,0,0,2,0,0,1,0,0,0,0
761,0,0,0,0,0,0,0,0,0,0,...,0,0,2,0,0,0,0,0,0,0
762,0,0,0,0,0,0,0,0,0,0,...,0,0,2,0,0,0,0,0,0,0


In [12]:
y_train = meta_df[['Superpopulation name_African Ancestry', 'Superpopulation name_American Ancestry', 'Superpopulation name_East Asian Ancestry', 'Superpopulation name_European Ancestry']]
y_train

,Superpopulation name_African Ancestry,Superpopulation name_American Ancestry,Superpopulation name_East Asian Ancestry,Superpopulation name_European Ancestry
0,False,False,True,False
1,False,True,False,False
2,True,False,False,False
3,False,False,True,False
4,False,False,False,True
...,...,...,...,...
759,True,False,False,False
760,False,False,False,True
761,False,False,False,True
762,False,False,False,True


## Wrap it in MultiOutputClassifier for multi-label classification and train the Model

In [13]:
model = MultiOutputClassifier(rfclass)

# Train the model
model.fit(X_train, y_train)


MultiOutputClassifier(estimator=RandomForestClassifier())

# Process the Validation Data

In [14]:
validation_set = validation_test.merge(pop_df, on='Population code', how='inner') 

X_val = validation_set.select_dtypes(include=['number']).drop(columns=['Population latitude','Population longitude','Superpopulation display order'])
y_val = pd.get_dummies(validation_set['Superpopulation name'])

# Make predictions on X_val
y_pred = model.predict(X_val)

In [15]:
print(y_val.shape)
print(y_pred.shape)

(164, 4)
(164, 4)


In [16]:
# Evaluate the model
print("\nClassification Report:")
print(classification_report(y_val, y_pred))

# Calculate accuracy if needed
accuracy = accuracy_score(y_val, y_pred)
print(f"Overall Accuracy: {accuracy:.4f}")


Classification Report:
              precision    recall  f1-score   support

           0       1.00      0.92      0.96        36
           1       0.83      0.36      0.50        28
           2       1.00      1.00      1.00        43
           3       0.90      0.98      0.94        57

   micro avg       0.95      0.87      0.90       164
   macro avg       0.93      0.81      0.85       164
weighted avg       0.94      0.87      0.88       164
 samples avg       0.86      0.87      0.86       164

Overall Accuracy: 0.8598


/opt/anaconda3/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
